In [4]:
# 模擬選課推薦系統（含通識和選修課程）
from typing import Dict, List, Set

# 課程數據
courses = {
    # 原有核心課程
    "A": {
        "name": "基礎程式設計",
        "desc": "學習 Python 的基礎語法和簡單程式設計概念",
        "prereqs": set(),  # 無先修
        "topics": {"程式設計", "Python"},
        "type": "核心"
    },
    "B": {
        "name": "資料結構",
        "desc": "介紹基本的資料結構，如陣列、鏈結串列和樹",
        "prereqs": {"A"},  # 先修 A
        "topics": {"程式設計", "資料結構"},
        "type": "核心"
    },
    "C": {
        "name": "機器學習入門",
        "desc": "學習機器學習的基本概念和簡單模型，使用 Python 實作",
        "prereqs": {"A"},  # 先修 A
        "topics": {"機器學習", "Python", "AI"},
        "type": "核心"
    },
    "D": {
        "name": "進階程式設計",
        "desc": "深入探討物件導向程式設計和演算法",
        "prereqs": {"A"},  # 先修 A
        "topics": {"程式設計", "演算法"},
        "type": "核心"
    },
    "E": {
        "name": "人工智慧概論",
        "desc": "介紹 AI 的基礎理論和應用，不含實作",
        "prereqs": set(),  # 無先修
        "topics": {"AI", "理論"},
        "type": "核心"
    },
    # 新增通識課程
    "GE1": {
        "name": "科技與社會",
        "desc": "探討科技對社會的影響，涵蓋 AI 和數位化趨勢",
        "prereqs": set(),  # 無先修
        "topics": {"科技", "社會", "AI"},
        "type": "通識"
    },
    "GE2": {
        "name": "環境科學入門",
        "desc": "介紹環境問題與可持續發展的基本概念",
        "prereqs": set(),  # 無先修
        "topics": {"環境", "科學"},
        "type": "通識"
    },
    # 新增選修課程
    "EL1": {
        "name": "網頁開發基礎",
        "desc": "學習 HTML、CSS 和 JavaScript 打造簡單網頁",
        "prereqs": {"A"},  # 先修 A
        "topics": {"程式設計", "網頁開發"},
        "type": "選修"
    },
    "EL2": {
        "name": "深度學習概論",
        "desc": "介紹深度學習的基本原理和應用，使用 Python",
        "prereqs": {"C"},  # 先修 C
        "topics": {"機器學習", "AI", "Python"},
        "type": "選修"
    },
    "EL3": {
        "name": "遊戲設計入門",
        "desc": "學習遊戲設計基礎，包含程式設計與創意發想",
        "prereqs": {"A"},  # 先修 A
        "topics": {"程式設計", "遊戲設計"},
        "type": "選修"
    }
}



In [ ]:
# 檢查先修條件是否滿足
def check_prereqs(course_id: str, completed: Set[str]) -> bool:
    return courses[course_id]["prereqs"].issubset(completed)

# 模擬 LLM 解析興趣（簡單關鍵詞匹配）
def parse_interests(input_text: str) -> Set[str]:
    interest_map = {
        "AI": {"AI", "機器學習"},
        "程式設計": {"程式設計", "Python", "演算法"},
        "Python": {"Python"},
        "資料結構": {"資料結構"},
        "理論": {"理論"},
        "科技": {"科技"},
        "社會": {"社會"},
        "環境": {"環境"},
        "網頁": {"網頁開發"},
        "遊戲": {"遊戲設計"}
    }
    interests = set()
    for keyword, topics in interest_map.items():
        if keyword in input_text:
            interests.update(topics)
    return interests

# 推薦課程
def recommend_courses(completed: Set[str], interests: str) -> List[str]:
    # 解析興趣
    interest_topics = parse_interests(interests)
    
    # 找到可選課程
    available_courses = []
    for course_id, info in courses.items():
        if check_prereqs(course_id, completed):
            # 計算課程與興趣的相關性（主題交集大小）
            relevance = len(info["topics"] & interest_topics)
            if relevance > 0:  # 只推薦與興趣相關的課程
                available_courses.append((course_id, relevance, info["type"]))
    
    # 按相關性排序，並優先核心課程，然後選修、通識
    available_courses.sort(key=lambda x: (x[1], {"核心": 0, "選修": 1, "通識": 2}[x[2]]), reverse=True)
    
    # 生成推薦清單
    recommendations = []
    for course_id, relevance, course_type in available_courses[:10]:  # 最多推薦 3 門
        course = courses[course_id]
        reason = f"推薦 '{course['name']}' ({course_type}) 因為它與你的興趣 '{interests}' 相關，且先修條件已滿足。"
        recommendations.append(reason)
    
    return recommendations

In [9]:
# 主程式
def main():
    # 用戶輸入（模擬）
    completed_courses = {"A"}  # 已修過基礎程式設計
    user_interests = "AI"
    
    # 獲取推薦
    recommendations = recommend_courses(completed_courses, user_interests)
    
    # 輸出結果
    print("根據你的背景和興趣：")
    for rec in recommendations:
        print(rec)
    if not recommendations:
        print("目前沒有符合你興趣且先修條件滿足的課程。")

if __name__ == "__main__":
    main()

根據你的背景和興趣：
推薦 '機器學習入門' (核心) 因為它與你的興趣 'AI' 相關，且先修條件已滿足。
推薦 '科技與社會' (通識) 因為它與你的興趣 'AI' 相關，且先修條件已滿足。
推薦 '人工智慧概論' (核心) 因為它與你的興趣 'AI' 相關，且先修條件已滿足。
